In [ ]:
## import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from time import time

from imblearn.ensemble import BalanceCascade 
import csv

In [ ]:
def get_subdata(data, size=10000):
    column = ['KORREKTUR']
    if data.KORREKTUR.dtype != np.float32:
        korrektur_vals = data[column].apply(lambda x: x.str.replace(',', '.'))[column[0]]
    else:
        korrektur_vals = data[column[0]]
    korrektur_vals = pd.to_numeric(korrektur_vals, downcast='float', errors='ignore')

    korrektur_df = pd.concat([data.ID, korrektur_vals], axis=1)

    target =  korrektur_df.drop_duplicates(subset=['ID'])[['ID', 'KORREKTUR']].reset_index(drop=True)
    N = len(target)

    ratio = np.sign(target.KORREKTUR).mean()
    n_samples = int(ratio * size)
    indices1 = np.random.choice(target[target.KORREKTUR>0].ID, size = n_samples, replace=False)
    indices0 = np.random.choice(target[target.KORREKTUR==0].ID, size = size - n_samples, replace=False)

    indices = np.append(indices0, indices1)
    return data.set_index('ID').loc[indices].reset_index()

In [ ]:
## import data
DATA = pd.read_csv("Data/daten_anonym2/arzta_daten_anonym1.csv", sep=';')
print(DATA.shape)

# DATA = get_subdata(DATA, size=10000)
# print(DATA.shape)

In [ ]:
DATA.head()

* **Same** for each ID: *RECHNUNGSBETRAG, ALTER, GESCHLECHT, VERSICHERUNG == FACHRICHTUNG* (4)
* **Diff** for each ID: *NUMMER, NUMMER_KAT, ANZAHL, FAKTOR, BETRAG, ART, TYP, LEISTUNG* (8)

In [ ]:
%%time 
## transfrom str type to float type
columns_comma = ['RECHNUNGSBETRAG', 'FAKTOR', 'BETRAG', 'ALTER', 'KORREKTUR'] 
DATA[columns_comma] = DATA[columns_comma].apply(lambda x: x.str.replace(',', '.'))
for column in columns_comma:
    DATA[column] = pd.to_numeric(DATA[column], downcast='float')

In [ ]:
%%time
## construct target column
target = DATA.groupby(['ID'])['KORREKTUR'].apply(lambda dt: int(np.sign(dt).values[0])).to_frame(name='target')
DATA = DATA.merge(target, on='ID', how='inner')

In [ ]:
## DATA must be sorted by Id
current_id = ''
indices = []

for i, Id in enumerate(DATA.ID):
    if Id != current_id:
        if i > 0: 
            indices.append(current_index)
        current_id = Id
        current_index = [i]
    else:
        current_index.append(i)
        
indices.append(current_index)

### BETRAG, ANZAHL: mean, std, min, max, median

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(121)
sns.distplot(DATA.BETRAG)
plt.subplot(122)
sns.distplot(np.log(DATA.BETRAG  - DATA.BETRAG.min() + 1));

In [ ]:
# BETRAG / ANZAHL (prices for one treatment) 
vals = DATA.BETRAG / DATA.ANZAHL
DATA.BETRAG = np.log(vals - np.min(vals, 0) +  1)

In [ ]:
%%time
# construct mean and std values for each ID
betrag_mean = DATA.groupby(['ID'])['BETRAG'].mean()
betrag_std = DATA.groupby(['ID'])['BETRAG'].std().fillna(0)
betrag_min = DATA.groupby(['ID'])['BETRAG'].min()
betrag_max = DATA.groupby(['ID'])['BETRAG'].max()
betrag_median = DATA.groupby(['ID'])['BETRAG'].median()

betrag_all = pd.concat([betrag_mean, betrag_std, betrag_min, betrag_max, betrag_median], axis=1, keys=
          ['BETRAG_mean', 'BETRAG_std', 'BETRAG_min', 'BETRAG_max', 'BETRAG_median']).reset_index()

In [ ]:
betrag_all.head()

In [ ]:
%%time
betrag_data = []
start = time()
ticks = np.linspace(2, 6, 100)
ids = DATA.ID.unique()
for i, Id in enumerate(ids):
    indices[i]
    betrag_data.append(np.histogram(DATA.iloc[indices[i]]['BETRAG'], ticks)[0])
    if i % 3000 == 0: 
        print(i, 'Done', np.round(i / len(ids) * 100, 2) , '% time:', time() - start)    

betrag_data = pd.DataFrame(betrag_data, columns = ['betrag' + str(np.round(val, 2)) for val in ticks[:-1]])
betrag_data = pd.concat([pd.Series(ids, name='ID'), betrag_data], axis=1)

In [ ]:
betrag_data.head()

### FAKTOR: mean, std, min, max, median

In [ ]:
%%time
# construct mean and std values for each ID
faktor_mean = DATA.groupby(['ID'])['FAKTOR'].mean()
faktor_std = DATA.groupby(['ID'])['FAKTOR'].std().fillna(0)
faktor_min = DATA.groupby(['ID'])['FAKTOR'].min()
faktor_max = DATA.groupby(['ID'])['FAKTOR'].max()
faktor_median = DATA.groupby(['ID'])['FAKTOR'].median()

faktor_all = pd.concat([faktor_mean, faktor_std, faktor_min, faktor_max, faktor_median], axis=1, keys=
          ['FAKTOR_mean', 'FAKTOR_std', 'FAKTOR_min', 'FAKTOR_max', 'FAKTOR_median']).reset_index()

In [ ]:
faktor_all.head()

### TYP: mean, std, min, max, median

In [ ]:
%%time
# construct mean and std values for each ID
DATA.TYP.fillna(-1, inplace=True)
typ_mean = DATA.groupby(['ID'])['TYP'].mean()
typ_std = DATA.groupby(['ID'])['TYP'].std().fillna(0)
typ_min = DATA.groupby(['ID'])['TYP'].min()
typ_max = DATA.groupby(['ID'])['TYP'].max()
typ_median = DATA.groupby(['ID'])['TYP'].median()

typ_all = pd.concat([typ_mean, typ_std, typ_min, typ_max, typ_median], axis=1, keys=
          ['TYP_mean', 'TYP_std', 'TYP_min', 'TYP_max', 'TYP_median']).reset_index()

In [ ]:
typ_all.head()

### NUMMER, NUMMER_KAT, LEISTUNG, ART. Encode all treatments and their categories

In [ ]:
%%time

nummer_list = []

ids = DATA.ID.unique()
start = time()
for i, Id in enumerate(ids):
    nummer_counter = DATA.iloc[indices[i]]['NUMMER'].value_counts()
    nummer_list.append(dict(nummer_counter))
    if i % 3000 == 0: 
        print(i, 'Done', np.round(i / len(ids) * 100, 2) , '% time:', time() - start)        

In [ ]:
# batch_size = 100
# nummer_list_size = int(len(nummer_list) / (batch_size - 1))

# start = time()
# for i in range(batch_size):
#     batch = nummer_list[i * nummer_list_size : (i + 1) * nummer_list_size]
#     if i == 0:    
#         pd.DataFrame(batch, columns=DATA.NUMMER.unique()).to_csv('NummerData.csv', index=False)
#     else:
#         with open('NummerData.csv', 'a') as csvFile:
#             writer = csv.writer(csvFile)
#             writer.writerows(pd.DataFrame(batch, columns=DATA.NUMMER.unique()).values)

#         csvFile.close()
#     print(i, 'Done,', ' time:', time() - start)

In [ ]:
# %%time
# chunksize = 1000
# chunks = pd.read_csv('NummerData.csv', chunksize=chunksize)

# nummer_data = []
# start = time()
# for i, chunk in enumerate(chunks):
#     nummer_data.append(chunk.to_sparse())
#     print(i, 'Done,', ' time:', time() - start)
    
# nummer_df = pd.concat(nummer_data)

# nummer_df = pd.read_csv('NummerData.csv')
# nummer_df = nummer_df.to_sparse()

In [ ]:
%%time
## Attention: it takes a lot of memory

nummer_df = pd.DataFrame(nummer_list).to_sparse()

In [ ]:
%%time

nummer_kat_list = []
leistung_list = []
art_list = []

start = time()
ids = DATA.ID.unique()
for i, Id in enumerate(ids):

    nummer_kat_counter = DATA.iloc[indices[i]]['NUMMER_KAT'].value_counts()
    leistung_counter = DATA.iloc[indices[i]]['LEISTUNG'].value_counts()
    art_counter = DATA.iloc[indices[i]]['ART'].value_counts()
    
    nummer_kat_list.append(dict(nummer_kat_counter))
    leistung_list.append(dict(leistung_counter))
    art_list.append(dict(art_counter))
    
    if i % 3000 == 0: 
        print(i, 'Done', np.round(i / len(ids) * 100, 2) , '% time:', time() - start)    

In [ ]:
%%time 
nummer_kat_df = pd.DataFrame(nummer_kat_list).to_sparse()
leistung_df = pd.DataFrame(leistung_list).to_sparse()
art_df = pd.DataFrame(art_list).to_sparse()

## Construct new DataFrame

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve, roc_curve

def plot_confusion_matrix(prediction, y_test):
    plt.figure(figsize=(15, 6))
    c_matrix = confusion_matrix(y_test, prediction)
    c_matrix_ = np.round(100 * c_matrix / c_matrix.sum(axis=1).reshape(-1, 1))
    plt.subplot(121)
    sns.heatmap(c_matrix, annot=True, fmt="d");
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.subplot(122)
    sns.heatmap(np.asarray(c_matrix_, dtype=int), annot=True, fmt="d")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def get_roc_auc_score(prediction, y_test):
    value = np.round(roc_auc_score(y_test, prediction[:,1]),4)
    print('ROC AUC score:',  value)
    
    return value

def get_pr_auc_score(prediction, y_test):
    value = np.round(average_precision_score(y_test, prediction[:, 1]),4)
    print('PR AUC score:',  value)
    
    return value
    
def plot_curves(prediction, y_test):
    tpr, fpr, _ = roc_curve(y_test, prediction[:,1])
    roc_auc = roc_auc_score(y_test, prediction[:,1])
    
    precision, recall, _ = precision_recall_curve(y_test, prediction[:,1])
    average_precision = average_precision_score(y_test, prediction[:,1])

    plt.figure(figsize=(15, 6))
    plt.subplot(121)
    plt.step(tpr, fpr, color='b', alpha=0.2, where='post')
    plt.fill_between(tpr, fpr, step='post', alpha=0.2, color='b')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class ROC curve: ROC AUC={0:0.2f}'.format(roc_auc))

    plt.subplot(122)
    plt.step(recall, precision, color='b', alpha=0.2, where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
    
def get_threshold_by_f1(prediction_proba, y_test, plot=False):
    thresholds = np.linspace(prediction_proba[:, 1].min(), prediction_proba[:, 1].max(), 300)
    curve = []
    for val in thresholds:
       
        prediction = np.asarray((prediction_proba[:, 1] >= val), int)
        curve.append(f1_score(prediction, y_test))

    argmax = np.argmax(curve)

    if plot == True:
        plt.figure(figsize=(14, 4))
        plt.plot(thresholds, curve)
        plt.plot([thresholds[argmax]] * 2, [np.min(curve), np.max(curve)])
    
    return thresholds[argmax]

def plot_probas(prediction_proba, y_test):
    plt.figure(figsize=(14, 4))
    probas = prediction_proba[:, 1]
    ax = sns.distplot(probas)
    proba_mean = probas.mean()
    proba_f1_best = get_threshold_by_f1(prediction_proba, y_test)
    plt.plot([proba_mean] * 2, ax.get_ylim(), label='Mean of probabilites')
    plt.plot([proba_f1_best] * 2, ax.get_ylim(), label='Best threshold by f1 score')
    plt.legend(fontsize=16)

In [ ]:
data = DATA.drop_duplicates(subset=['ID'])[['ID', 'RECHNUNGSBETRAG', 'ALTER', 'GESCHLECHT', 'VERSICHERUNG', 'target']].reset_index(drop=True)

data = data.merge(betrag_all, on='ID', how='inner')
data = data.merge(betrag_data, on='ID', how='inner')
data = data.merge(typ_all, on='ID', how='inner')
data = data.merge(faktor_all, on='ID', how='inner')

data.head()

In [ ]:
!!!

In [ ]:
%%time 
# nummer_df.fillna(0, inplace=True)
# nummer_kat_df.fillna(0, inplace=True)
# leistung_df.fillna(0, inplace=True)
# art_df.fillna(0, inplace=True)

data = pd.concat([data, nummer_df], axis=1)
data = pd.concat([data, nummer_kat_df], axis=1)
data = pd.concat([data, leistung_df], axis=1)
data = pd.concat([data, art_df], axis=1)

In [ ]:
def save_csv(data):
    batch_size = 100
    data_size = int(len(data) / (batch_size - 1))

    start = time()
    for i in range(batch_size):
        batch = data.iloc[i * data_size : (i + 1) * data_size]
        if i == 0:    
            batch.to_csv('TransformedData.csv', index=False)
        else:
            with open('TransformedData.csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerows(batch.values)

            csvFile.close()
        print(i, 'Done,', ' time:', time() - start)
        
def read_csv(filename='TransformedData.csv'):
    chunksize = 10000
    chunks = pd.read_csv(filename, chunksize=chunksize)

    data = []
    start = time()
    for i, chunk in enumerate(chunks):
        data.append(chunk.to_sparse())
        print(i, 'Done,', ' time:', time() - start)

    return pd.concat(data)

In [ ]:
save_csv(data)

In [ ]:
pd.read_csv('TransformedData.csv')
#data = data.to_sparse()